# Run analysis on 90 deg fine resolution grid scan

In [1]:
%matplotlib qt

In [2]:
%run -i ~/Documents/tomology/tomography/utils.py

/home/xf28id2/conda_envs/khalifah_20210725/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [3]:
import pyFAI
import pathlib

from ipywidgets import interact_manual

## Create the calculator

In [4]:
C = Calculator()

## Load the Run Data

In [5]:
from databroker import catalog

db = catalog["xpdd"]

In [7]:
UID = '1854f609-2ea8-4bc5-9b5e-31705400a609'
run = db[UID]
C.metadata = dict(run.metadata["start"])

In [8]:
data = run.primary.to_dask()
C.frames_arr = data["dexela_image"]
C.frames_arr

,Array,Chunk
Bytes,229.24 GiB,91.12 MiB
Shape,"(2576, 1, 3888, 3072)","(1, 1, 3888, 3072)"
Count,7728 Tasks,2576 Chunks
Type,float64,numpy.ndarray


In [9]:
C.metadata["shape"] = C.metadata["shape"][1:]
C.metadata["extents"] = C.metadata["extents"][1:]

In [10]:
C.calc_coords()

In [13]:
# show the total intensity distribution
total_i = data["dexela_stats1_total"].values.reshape(C.metadata["shape"])
total_i = xr.DataArray(total_i, coords={"x": C.coords[1], "y": C.coords[0]}, dims=["y", "x"])
fig, ax = plt.subplots()
total_i.plot.imshow(ax=ax)
ax.invert_yaxis()
ax.set_aspect("equal")
plt.show()

## Load the calibration data

In [27]:
PONI_FILE = "/nsls2/data/xpd/legacy/processed/from_mehmet/CeO2_focused_beam.poni"
C.ai = pyFAI.load(PONI_FILE) if PONI_FILE else pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=200, wavelength=0.186, detector="perkin")

## Choose the Max image

In [16]:
interact_manual(
    show_npy_array, 
    template="/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_1854f609-2ea8-4bc5-9b5e-31705400a609/max/CG-0046_{}_max.npy",
    index=(1200, 1888)
)

interactive(children=(Text(value='/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_1854f609-2ea8-4bc…

<function __main__.show_npy_array(template: str, index: int, **kwargs)>

In [17]:
INDEX = 1595
sample_name = C.metadata["sample_name"]
uid = C.metadata["uid"]
light_file = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/{}_{}/max/{}_{}_max.npy".format(sample_name, uid, sample_name, INDEX)
dark_file = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/{}_{}/min/{}_{}_min.npy".format(sample_name, uid, sample_name, INDEX)
C.light = np.load(light_file)
C.dark = np.load(dark_file)

## Subtract the dark from the light

In [18]:
C.light -= C.dark

## Locate the peaks and create windows

In [8]:
# tp.locate?

In [22]:
C.calc_peaks_from_light_frame(31, separation=100, threshold=20, topn=100)

In [23]:
C.calc_windows_from_peaks(num=100, width=40)
C.show_windows(size=10, vmax=300)
plt.show()

## Run throught the images to calculate the intensity in the windows

In [24]:
C.calc_intensity_in_windows()

In [ ]:
C.reshape_intensity()
C.assign_q_values()

## Save the result

In [33]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}_2.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
C.to_dataset().to_netcdf(FILENAME)

## Visualize the result

In [34]:
import matplotlib as mpl

In [36]:
mean = C.intensity.mean()
std = C.intensity.std()
facet = C.show_intensity(col_wrap=10, vmin=mean-2*std, vmax=mean+4*std, add_colorbar=False)
axes = facet.axes.flatten()
axes[0].invert_yaxis()
for i in range(C.intensity.shape[0]):
    d = 2 * np.pi / C.windows["Q"]
    axes[i].set_title("d = {:.3f} nm".format(d.iloc[i]))
    axes[i].set_aspect("equal")
plt.show()